# GuitarBot - Music Store Order Collection

This project contains the code to set up an automated service for collecting orders for a music store using OpenAI's GPT-3.5-turbo. The service, named GuitarBot, interacts with customers to take orders for guitars and related accessories.

### 1. Setting Up the Environment for OpenAI API

Configure the environment to use the OpenAI API by importing necessary libraries and loading the API key from a `.env` file.

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # Read local .env file

openai.api_key = os.getenv('OPENAI_API_KEY')

### 2. Defining Functions to Interact with OpenAI's API

Define two functions, `get_completion` and `get_completion_from_messages`, which facilitate interaction with the OpenAI API to generate responses based on user prompts.


In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
    )
    # print(str(response.choices[0].message))
    return response.choices[0].message["content"]

### 3. Generating and Printing a Response

Demonstrate how to use the `get_completion_from_messages` function to generate a response from the OpenAI API and print it.

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

### 4. Defining the `collect_messages` Function

Handle user input, generates a response using the OpenAI API, and updates the display with the conversation history.

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role': 'user', 'content': f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

### 5. Setting Up the GUI for GuitarBot

Set up the graphical user interface (GUI) using the Panel library. This includes initializing the interface, defining the conversation context, and creating interactive widgets for user input and responses.

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = []  # collect display
context = [{'role': 'system', 'content': """
You are GuitarBot, an automated service to collect orders for a music store. \
You first greet the customer, then collect the order, \
and then ask if it's for pickup or delivery. \
You wait to collect the entire order, then summarize it and check one last \
time if the customer wants to add anything else. \
If it's for delivery, you ask for an address. \
Finally, you collect the payment. \
Make sure to clarify all options, extras, and models to uniquely \
identify the item from the catalog. \
You respond in a short, very conversational friendly style. \
The catalog includes \
Fender Stratocaster guitar 1200.00, 1000.00, 800.00 \
Gibson Les Paul guitar 1500.00, 1300.00, 1100.00 \
Ibanez RG guitar 900.00, 750.00, 600.00 \
Suhr Classic guitar 2000.00, 1800.00, 1600.00 \
PRS Custom 24 guitar 2500.00, 2300.00, 2100.00 \
Fender amplifier 500.00, 300.00 \
Marshall amplifier 600.00, 400.00 \
Picks (pack of 12) 5.00 \
Capo 10.00 \
Guitar cable 15.00 \
Guitar strap 20.00 \
"""}]  # accumulate messages

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")
interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)
dashboard


### 6. Creating a JSON Summary of the Order

Create a message to request a JSON summary of the previous musical instrument order and print the response.

In [ ]:
messages = context.copy()
messages.append(
    {'role': 'system', 'content': 'create a JSON summary of the previous musical instrument order. Itemize the price for each item.\
    The fields should be 1) guitar, include model and price 2) list of accessories and prices 3) total price'},
)

response = get_completion_from_messages(messages, temperature=0)
print(response)